## Tutorial 
- Setup a SCM using symbolic
- https://github.com/maichmueller/scmodels

It is based on SymPy under the hood:
- https://docs.sympy.org/latest/index.html
- https://docs.sympy.org/latest/modules/stats.html#random-variable-types

## this small section test out the Finite and Discrete type
- the Multinomial causes kernel crash

In [1]:
import pandas as pd
from scmodels import SCM

In [2]:
discretescm = SCM(
    [
        "DiscreteUniform = N, N ~ DiscreteUniform(items='1 2 3')", # items are the choices, has to be integers
        "Die             = N, N ~ Die(sides=6)",
        "Bernoulli       = N, N ~ Bernoulli(p=0.8)", # binary outcome
        #"D4 = N, N ~ Coin()",
        "Binomial        = N, N ~ Binomial(n=2, p=0.5, succ=2, fail=4)",  # n is number of trials, determine the range
        "BetaBinomial    = N, N ~ BetaBinomial(n=10, alpha=0.5, beta=2.2)", # n is number of trials, determine the range
        
        # the first parameter is the number of trials
        # followed by a set of probability values that correspond the ordinality, e.g. 3 prob for ordinality of 3
        # the probability value set should sum up to one
        # THIS KEEPS CRASHING THE KERNEL!!
        #"Multinomial     = N, N ~ Multinomial(2, 0.5, 0.3, 0.2)",  
        
        "Geometric       = N, N ~ Geometric(p=0.8)",
        "Poisson         = N, N ~ Poisson(lamda=0.8)",
        "Logarithmic     = N, N ~ Logarithmic(p=0.5)",
        
        "A               = N, N ~ DiscreteUniform(items='1 0 2')",
        "B               = N, N ~ Bernoulli(p=0.8)",
        "C               = A*B ",
    ]
)

In [3]:
discretescm.sample(500).round(1).describe().round(0)

C:\Users\uvgupta\software\miniconda3\envs\cdt\lib\site-packages\scmodels\scm.py:286: SymPyDeprecationWarning: 

The numsamples parameter to sympy.stats.sample() is deprecated.
Either use a list comprehension, like

[sample(...) for i in range(500)]

or add a dimension to size, like

sample(..., size=(500,))

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympy-stats-numsamples
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  list(sample(noise_gen, numsamples=n, seed=seed)), dtype=float


,DiscreteUniform,Die,Bernoulli,Binomial,BetaBinomial,Geometric,Poisson,Logarithmic,A,B,C
count,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0
mean,2.0,4.0,1.0,6.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
std,1.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0
min,1.0,1.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
25%,1.0,2.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
50%,2.0,4.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,3.0,5.0,1.0,6.0,3.0,1.0,1.0,2.0,2.0,1.0,2.0
max,3.0,6.0,1.0,8.0,10.0,6.0,6.0,11.0,2.0,1.0,2.0


### Run a tutorial from SymPy document
- focus on MultiNomial problem
https://docs.sympy.org/latest/modules/stats.html#sympy.stats.Multinomial

In [4]:
from sympy.stats import density, Multinomial, marginal_distribution
from sympy import symbols
x1, x2, x3 = symbols('x1, x2, x3', nonnegative=True, integer=True)
p1, p2, p3 = symbols('p1, p2, p3', positive=True)
M = Multinomial('M', 30, p1, p2, p3)
density(M)(x1, x2, x3)
marginal_distribution(M, M[0])(x1).subs(x1, 1)

30*p1*p2**29 + 870*p1*p2**28*p3 + 12180*p1*p2**27*p3**2 + 109620*p1*p2**26*p3**3 + 712530*p1*p2**25*p3**4 + 3562650*p1*p2**24*p3**5 + 14250600*p1*p2**23*p3**6 + 46823400*p1*p2**22*p3**7 + 128764350*p1*p2**21*p3**8 + 300450150*p1*p2**20*p3**9 + 600900300*p1*p2**19*p3**10 + 1037918700*p1*p2**18*p3**11 + 1556878050*p1*p2**17*p3**12 + 2035917450*p1*p2**16*p3**13 + 2326762800*p1*p2**15*p3**14 + 2326762800*p1*p2**14*p3**15 + 2035917450*p1*p2**13*p3**16 + 1556878050*p1*p2**12*p3**17 + 1037918700*p1*p2**11*p3**18 + 600900300*p1*p2**10*p3**19 + 300450150*p1*p2**9*p3**20 + 128764350*p1*p2**8*p3**21 + 46823400*p1*p2**7*p3**22 + 14250600*p1*p2**6*p3**23 + 3562650*p1*p2**5*p3**24 + 712530*p1*p2**4*p3**25 + 109620*p1*p2**3*p3**26 + 12180*p1*p2**2*p3**27 + 870*p1*p2*p3**28 + 30*p1*p3**29

## Do an interventional at A
- then sample from it

In [7]:
discretescm.do_intervention([("A", 1)])
discretescm.sample(10)

C:\Users\uvgupta\software\miniconda3\envs\cdt\lib\site-packages\scmodels\scm.py:286: SymPyDeprecationWarning: 

The numsamples parameter to sympy.stats.sample() is deprecated.
Either use a list comprehension, like

[sample(...) for i in range(10)]

or add a dimension to size, like

sample(..., size=(10,))

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympy-stats-numsamples
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  list(sample(noise_gen, numsamples=n, seed=seed)), dtype=float


,DiscreteUniform,Die,Bernoulli,Binomial,BetaBinomial,Geometric,Poisson,Logarithmic,A,B,C
0,3.0,5.0,0.0,6.0,3.0,1.0,2.0,4.0,1,0.0,0.0
1,3.0,2.0,1.0,8.0,0.0,2.0,2.0,1.0,1,1.0,1.0
2,1.0,4.0,1.0,6.0,0.0,1.0,1.0,1.0,1,0.0,0.0
3,3.0,6.0,1.0,6.0,0.0,1.0,0.0,1.0,1,1.0,1.0
4,3.0,1.0,1.0,6.0,4.0,1.0,0.0,1.0,1,1.0,1.0
5,1.0,2.0,1.0,8.0,4.0,1.0,1.0,1.0,1,0.0,0.0
6,2.0,3.0,1.0,8.0,2.0,2.0,1.0,1.0,1,1.0,1.0
7,1.0,6.0,1.0,6.0,4.0,1.0,0.0,1.0,1,1.0,1.0
8,3.0,4.0,1.0,6.0,7.0,2.0,1.0,1.0,1,0.0,0.0
9,3.0,2.0,0.0,6.0,0.0,1.0,2.0,1.0,1,1.0,1.0


In [ ]:
#myscm.undo_intervention()